**Drive de google.colab**

In [1]:
# Montar Drive (si no está montado)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importación y configuración.**

In [2]:
# === IMPORTS ===
import os
import numpy as np
import pandas as pd

from statsmodels.tsa.statespace.structural import UnobservedComponents
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import STL

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Ruta de origen y lectura
ruta = "/content/drive/MyDrive/Proyecto Integrador TEC 2025/EDA/nacional_semanal.csv"
df = pd.read_csv(ruta)

**Visualización previa**

In [4]:
df.head(5)

,Fecha,Nuevos_Casos
0,2014-01-05,11.0
1,2014-01-12,89.0
2,2014-01-19,127.0
3,2014-01-26,114.0
4,2014-02-02,159.0


**Configuración y series de trabajo**

In [5]:
S = 52  # periodicidad semanal

# Copia y ordena el DataFrame original
df_model = df.copy().sort_values("Fecha").reset_index(drop=True)

# Serie objetivo
y = df_model.set_index("Fecha")["Nuevos_Casos"].astype(float)

# Asegura índice tipo datetime y orden temporal
y.index = pd.to_datetime(y.index)
y = y.sort_index()

# División temporal (train / test)
TEST_WEEKS = 52
cut = max(len(y) - TEST_WEEKS, 1)
y_train, y_test = y.iloc[:cut], y.iloc[cut:]

# Asegurar que ambos índices son datetime
y_train.index = pd.to_datetime(y_train.index)
y_test.index = pd.to_datetime(y_test.index)

# === Verificación
print(f"Tamaño total: {len(y)} | Train: {len(y_train)} | Test: {len(y_test)}")
print(f"Train desde {y_train.index.min().date()} hasta {y_train.index.max().date()}")
print(f"Test  desde {y_test.index.min().date()} hasta {y_test.index.max().date()}")


Tamaño total: 573 | Train: 521 | Test: 52
Train desde 2014-01-05 hasta 2023-12-24
Test  desde 2023-12-31 hasta 2024-12-22


**Métricas (MAE, sMAPE, MASE)**

In [6]:
def mae(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    return float(np.mean(np.abs(y_true[mask] - y_pred[mask])))

# Métricas
def smape(y_true, y_pred, eps=1e-9):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    den = np.maximum(np.abs(y_true) + np.abs(y_pred), eps)  # evita división por 0
    return float(200.0 * np.mean(np.abs(y_pred - y_true) / den))

def mase(y_train, y_true, y_pred, m=52):  # semanal: m=52
    y_train = np.asarray(y_train, dtype=float)
    y_true  = np.asarray(y_true,  dtype=float)
    y_pred  = np.asarray(y_pred,  dtype=float)

    # limpiar
    y_train = y_train[np.isfinite(y_train)]

    # escala con error del naïve estacional (TRAIN)
    if len(y_train) > m:
        diff = y_train[m:] - y_train[:-m]
        diff = diff[np.isfinite(diff)]
        scale = np.mean(np.abs(diff)) if diff.size else np.nan
    else:
        # fallback si la serie es corta
        yt = y_train
        scale = np.mean(np.abs(np.diff(yt))) if yt.size > 1 else np.nan

    if not np.isfinite(scale) or scale == 0:
        return float('nan')

    # MAE del modelo en el segmento de evaluación
    mask_ev = np.isfinite(y_true) & np.isfinite(y_pred)
    mae_model = np.mean(np.abs(y_true[mask_ev] - y_pred[mask_ev])) if np.any(mask_ev) else np.nan

    return float(mae_model / scale) if np.isfinite(mae_model) else float('nan')

**Fourier (para estacionalidad flexible en ARIMA/SARIMAX)**

In [7]:
def fourier(df_index, m=S, K=2):
    """Devuelve DataFrame con senos/cosenos para K armónicos."""
    t = np.arange(len(df_index))
    feats = {}
    for k in range(1, K+1):
        feats[f'sin_{k}'] = np.sin(2*np.pi*k*t/m)
        feats[f'cos_{k}'] = np.cos(2*np.pi*k*t/m)
    return pd.DataFrame(feats, index=df_index)

**Interfaz base y registro de modelos**

In [8]:
class BaseForecaster:
    def fit(self, y_train: pd.Series, exog: pd.DataFrame | None = None): ...
    def predict(self, steps: int, exog: pd.DataFrame | None = None) -> pd.Series: ...

REGISTRY = {}  # nombre -> clase
def register(name):
    def _wrap(cls):
        REGISTRY[name] = cls
        return cls
    return _wrap

**ETS (Holt–Winters amortiguado)**

In [9]:
@register("ETS")
class ETSForecaster(BaseForecaster):
    def __init__(self, trend="add", damped_trend=True, seasonal=None, seasonal_periods=S):
        self.kw = dict(trend=trend, damped_trend=damped_trend,
                       seasonal=seasonal, seasonal_periods=seasonal_periods)
        self.model_ = None
    def fit(self, y_train):
        self.model_ = ExponentialSmoothing(y_train, **self.kw).fit(optimized=True)
        self.index_train_ = y_train.index
        return self
    def predict(self, steps):
        idx = pd.date_range(self.index_train_[-1] + pd.offsets.Week(weekday=6),
                            periods=steps, freq="W-SUN")
        fc = self.model_.forecast(steps)
        return pd.Series(fc.values, index=idx, name="yhat")

**SARIMA (SARIMAX sin exógenas)**

In [10]:
@register("SARIMA")
class SARIMAForecaster(BaseForecaster):
    def __init__(self, order=(1,1,1), seasonal_order=(0,0,0,52), transform=None):
        self.order = order
        self.seasonal_order = seasonal_order
        self.transform = transform
        self.res_ = None
        self.endog_index_ = None
        self.freq_ = None
        self.exog_cols_ = None  # columnas esperadas de exógenas con residuos

    def _apply_transform(self, y):
        if self.transform == "log":
            return np.log(y)
        elif self.transform == "log1p":
            return np.log1p(y)
        return y

    def _invert_transform(self, arr):
        if self.transform == "log":
            return np.exp(arr)
        elif self.transform == "log1p":
            return np.expm1(arr)
        return arr

    def fit(self, y_train, exog: pd.DataFrame | None = None):
        y_train = pd.Series(y_train.astype(float), index=pd.to_datetime(y_train.index))
        y_t = self._apply_transform(y_train)

        # Guarda índice y frecuencia para construir horizontes coherentes
        self.endog_index_ = y_train.index
        self.freq_ = self.endog_index_.freq or pd.infer_freq(self.endog_index_)
        if self.freq_ is None:
            # fallback razonable semanal
            self.freq_ = "W-SUN"

        # Alinear exógenas al TRAIN
        if exog is not None:
            exog = exog.reindex(self.endog_index_).fillna(0)
            self.exog_cols_ = list(exog.columns)
        else:
            self.exog_cols_ = None

        mod = SARIMAX(
            y_t,
            exog=exog,
            order=self.order,
            seasonal_order=self.seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        self.res_ = mod.fit(disp=False)
        return self

    def predict(self, steps: int, exog: pd.DataFrame | None = None):
        # Si el modelo fue entrenado con exógenas, garantizar que el futuro tenga mismas columnas
        if self.exog_cols_ is not None:
            if exog is None:
                # construir exógenas futuras en ceros
                future_idx = pd.date_range(
                    self.endog_index_[-1] + pd.tseries.frequencies.to_offset(self.freq_),
                    periods=steps,
                    freq=self.freq_,
                )
                exog = pd.DataFrame(0, index=future_idx, columns=self.exog_cols_)
            else:
                # asegurar columnas y orden
                exog = exog.reindex(columns=self.exog_cols_).fillna(0)
                # si no trae índice, lo generamos
                if exog.index.dtype != "datetime64[ns]":
                    future_idx = pd.date_range(
                        self.endog_index_[-1] + pd.tseries.frequencies.to_offset(self.freq_),
                        periods=steps,
                        freq=self.freq_,
                    )
                    exog.index = future_idx

        fc = self.res_.get_forecast(steps=steps, exog=exog)
        fc_values = fc.predicted_mean.values
        fc_values = self._invert_transform(fc_values)

        # índice futuro
        if exog is not None and isinstance(exog.index, pd.DatetimeIndex):
            idx = exog.index
        else:
            idx = pd.date_range(
                self.endog_index_[-1] + pd.tseries.frequencies.to_offset(self.freq_),
                periods=steps,
                freq=self.freq_,
            )
        return pd.Series(fc_values, index=idx, name="yhat")

**ARIMA + Fourier (estacionalidad flexible)**

In [11]:
@register("ARIMA_FOURIER")
class ARIMAFourierForecaster(BaseForecaster):
    def __init__(self, order=(1,1,1), K=2, m=S):
        self.order = order; self.K = K; self.m = m
        self.res_ = None; self.last_index_ = None
    def fit(self, y_train):
        X = fourier(y_train.index, m=self.m, K=self.K)
        self.last_index_ = y_train.index
        mod = SARIMAX(y_train, order=self.order, seasonal_order=(0,0,0,0),
                      exog=X, enforce_stationarity=False, enforce_invertibility=False)
        self.res_ = mod.fit(disp=False)
        return self
    def predict(self, steps):
        idx = pd.date_range(self.last_index_[-1] + pd.offsets.Week(weekday=6),
                            periods=steps, freq="W-SUN")
        Xf = fourier(idx, m=self.m, K=self.K)
        fc = self.res_.get_forecast(steps=steps, exog=Xf)
        return pd.Series(fc.predicted_mean.values, index=idx, name="yhat")

**UCM (tendencia + estacionalidad opcional)**

In [12]:
@register("UCM")
class UCMForecaster(BaseForecaster):
    def __init__(self, level="local linear trend", seasonal=S, irregular=True):
        self.level = level; self.seasonal = seasonal; self.irregular = irregular
        self.res_ = None; self.end_ = None
    def fit(self, y_train):
        self.end_ = y_train.index
        mod = UnobservedComponents(y_train, level=self.level,
                                   seasonal=self.seasonal, irregular=self.irregular)
        self.res_ = mod.fit(disp=False)
        return self
    def predict(self, steps):
        fc = self.res_.get_forecast(steps=steps)
        idx = pd.date_range(self.end_[-1] + pd.offsets.Week(weekday=6),
                            periods=steps, freq="W-SUN")
        return pd.Series(fc.predicted_mean.values, index=idx, name="yhat")

**STL + ARIMA (modelo sobre residuo; la estacionalidad se quita vía STL)**

In [13]:
@register("STL_ARIMA")
class STL_ARIMA_Forecaster(BaseForecaster):
    def __init__(self, period=S, order=(1,1,1)):
        self.period = period; self.order = order
        self.stl_ = None; self.res_ = None; self.last_index_ = None
        self.trend_ = None; self.seasonal_ = None
    def fit(self, y_train):
        self.last_index_ = y_train.index
        self.stl_ = STL(y_train, period=self.period, robust=True).fit()
        resid = y_train - self.stl_.seasonal
        self.trend_ = self.stl_.trend
        self.seasonal_ = self.stl_.seasonal
        mod = SARIMAX(resid, order=self.order, seasonal_order=(0,0,0,0),
                      enforce_stationarity=False, enforce_invertibility=False)
        self.res_ = mod.fit(disp=False)
        return self
    def predict(self, steps):
        # proyección: residuo + última estacionalidad conocida
        idx = pd.date_range(self.last_index_[-1] + pd.offsets.Week(weekday=6),
                            periods=steps, freq="W-SUN")
        resid_fc = self.res_.get_forecast(steps=steps).predicted_mean
        # repetir patrón estacional de longitud S
        seas_tail = pd.Series(self.seasonal_.values[-self.period:], index=self.seasonal_.index[-self.period:])
        seas_fc = pd.Series(np.resize(seas_tail.values, steps), index=idx)
        return pd.Series(resid_fc.values + seas_fc.values, index=idx, name="yhat")

**Función para ejecutar modelo por nombre + hiperparámetros**

In [14]:
def run_model(model_name: str, **kwargs):
    if model_name not in REGISTRY:
        raise ValueError(f"Modelo '{model_name}' no está registrado. Opciones: {list(REGISTRY)}")

    # Extrae exógenas del TRAIN
    exog_train = kwargs.pop("exog", None)

    # Instancia el forecaster con el resto de parámetros
    model = REGISTRY[model_name](**kwargs)

    # Fit
    used_exog = False
    if exog_train is not None:
        exog_aligned = exog_train.reindex(y_train.index).fillna(0)
        try:
            model = model.fit(y_train, exog=exog_aligned)
            used_exog = True
        except TypeError:
            model = model.fit(y_train)
    else:
        model = model.fit(y_train)

    # Horizonte
    steps = len(y_test)

    # Predict
    if used_exog:
        exog_future = pd.DataFrame(0, index=y_test.index, columns=exog_aligned.columns)
        try:
            yhat = model.predict(steps=steps, exog=exog_future)
        except TypeError:
            yhat = model.predict(steps=steps)
    else:
        yhat = model.predict(steps=steps)

    # Alinear por índice
    y_true = y_test.loc[yhat.index.intersection(y_test.index)]
    y_pred = yhat.loc[y_true.index]

    # Métricas sin residuales
    out = {
        "model": model_name,
        "params": kwargs,
        "MAE": mae(y_true, y_pred),
        "sMAPE": smape(y_true, y_pred),
        "MASE": mase(y_train, y_true, y_pred, m=S),
    }

    return out, y_pred

**Creación de exogenas residuales**

In [15]:
# creación de exógenas (shocks)
# Re-fit del modelo SARIMA base para obtener residuales
mdl_tmp = SARIMAForecaster(order=(2,0,1), seasonal_order=(0,1,1,S), transform="log1p").fit(y_train)
resid = pd.Series(mdl_tmp.res_.resid, index=mdl_tmp.endog_index_)

# Identificar las TOP-3 semanas con mayores outliers (|residuo|)
TOP_K = min(3, len(resid))
idx_out = resid.abs().sort_values(ascending=False).head(TOP_K).index

# Crear las variables dummy (exógenas) para esas semanas
X_train = pd.DataFrame(0, index=y_train.index, columns=[f"shock_{i}" for i in range(TOP_K)])
for j, ix in enumerate(idx_out):
    if ix in X_train.index:
        X_train.loc[ix, f"shock_{j}"] = 1

# Verificación
print("Exógenas creadas:", list(X_train.columns))
print(X_train.sum())  # número de semanas marcadas por cada dummy

Exógenas creadas: ['shock_0', 'shock_1', 'shock_2']
shock_0    1
shock_1    1
shock_2    1
dtype: int64


**Ejecución (uno a la vez)**

In [16]:
# ETS amortiguado sin estacionalidad
res_ets, fc_ets = run_model("ETS", trend="add", damped_trend=True, seasonal="add")
print(res_ets)

{'model': 'ETS', 'params': {'trend': 'add', 'damped_trend': True, 'seasonal': 'add'}, 'MAE': 18.874143930548932, 'sMAPE': 12.791726635342629, 'MASE': 0.45448341651319246}


In [17]:
res_sarima, fc_sarima = run_model("SARIMA", order=(2,0,1), seasonal_order=(0,1,1,52))
print(res_sarima)

{'model': 'SARIMA', 'params': {'order': (2, 0, 1), 'seasonal_order': (0, 1, 1, 52)}, 'MAE': 17.509618996937913, 'sMAPE': 11.82009299440506, 'MASE': 0.42162608767078513}


In [18]:
res_af, fc_af = run_model("ARIMA_FOURIER", order=(2,0,1), K=2, m=52)
print(res_af)

{'model': 'ARIMA_FOURIER', 'params': {'order': (2, 0, 1), 'K': 2, 'm': 52}, 'MAE': 17.683462515189536, 'sMAPE': 12.423820875155183, 'MASE': 0.42581218460871245}


In [19]:
res_ucm, fc_ucm = run_model("UCM", level="local level", seasonal=52)
print(res_ucm)

{'model': 'UCM', 'params': {'level': 'local level', 'seasonal': 52}, 'MAE': 17.876734724131925, 'sMAPE': 12.156827942507977, 'MASE': 0.4304661182737523}


In [20]:
res_stl, fc_stl = run_model("STL_ARIMA", period=52, order=(1,0,1))
print(res_stl)

{'model': 'STL_ARIMA', 'params': {'period': 52, 'order': (1, 0, 1)}, 'MAE': 28.61958403951618, 'sMAPE': 18.021442103712932, 'MASE': 0.6891505321421723}


**Mejor modelo con parametros ajustados**

In [21]:
res_sarima, fc_sarima = run_model("SARIMA", order=(2,0,1), seasonal_order=(0,1,1,52), transform="log1p"); print(res_sarima)

{'model': 'SARIMA', 'params': {'order': (2, 0, 1), 'seasonal_order': (0, 1, 1, 52), 'transform': 'log1p'}, 'MAE': 17.025649539188684, 'sMAPE': 11.577978082557411, 'MASE': 0.4099722561934329}


In [22]:
res_sarima, fc_sarima = run_model("SARIMA", order=(2,0,1), seasonal_order=(0,1,1,52), transform="log"); print(res_sarima)

{'model': 'SARIMA', 'params': {'order': (2, 0, 1), 'seasonal_order': (0, 1, 1, 52), 'transform': 'log'}, 'MAE': 17.02936613510958, 'sMAPE': 11.58273631712917, 'MASE': 0.41006175064775857}


In [23]:
res_sarima_x, fc_sarima_x = run_model("SARIMA", order=(2,0,1), seasonal_order=(0,1,1,S), transform="log1p", exog=X_train)
print(res_sarima_x)

{'model': 'SARIMA', 'params': {'order': (2, 0, 1), 'seasonal_order': (0, 1, 1, 52), 'transform': 'log1p'}, 'MAE': 16.822437938944002, 'sMAPE': 11.460050610235603, 'MASE': 0.4050789851293699}


**Proceso de optimización SARIMA con variables exógenas (“shocks”)**

Se realizó una búsqueda local de hiperparámetros para el modelo SARIMA, incluyendo variaciones en los órdenes autorregresivos y de medias móviles tanto regulares como estacionales, así como el tipo de transformación logarítmica aplicada a la serie (log y log1p).

In [24]:
res_sarima_x, fc_sarima_x = run_model("SARIMA", order=(3,0,2), seasonal_order=(0,1,1,52), transform="log1p", exog=X_train); print(res_sarima_x)

{'model': 'SARIMA', 'params': {'order': (3, 0, 2), 'seasonal_order': (0, 1, 1, 52), 'transform': 'log1p'}, 'MAE': 16.50228644727971, 'sMAPE': 11.256874560177387, 'MASE': 0.3973698384645574}


**Guardar pronósticos y métricas**

In [25]:
# === Exportar predicciones ===

# Carpeta de salida
out_dir = "/content/drive/MyDrive/Proyecto Integrador TEC 2025/EDA"
os.makedirs(out_dir, exist_ok=True)  # crea la carpeta si no existe

# Crear DataFrame de resultados
df_preds = pd.DataFrame({
    "Fecha": y_test.index,
    "y_true": y_test.values,
    "yhat_ETS": fc_ets.reindex(y_test.index).values,
    })

# Guardar CSV en Drive
out_path = os.path.join(out_dir, "predicciones_nacional.csv")
df_preds.to_csv(out_path, index=False)

print(f"[OK] Archivo guardado en: {out_path}")

[OK] Archivo guardado en: /content/drive/MyDrive/Proyecto Integrador TEC 2025/EDA/predicciones_nacional.csv


# Conclusiones de los análisis

#  Comparación de Modelos ETS (Semanal vs Mensual)

Se evaluaron modelos **ETS (Error–Trend–Seasonal)** con diferentes configuraciones de **tendencia**, **amortiguamiento** y **estacionalidad**, para dos frecuencias temporales: **semanal** y **mensual**.

---

##  Modelo Semanal

**Tipo:** ETS (Holt–Winters)  
**Parámetros comunes:** `trend='add'`, `damped_trend=True`  
**Estacionalidad evaluada:** Sin estacionalidad vs Aditiva (`m=52`)

| Configuración | seasonal | seasonal_periods | MAE | sMAPE (%) | MASE |
|---------------|-----------|------------------|-----|-----------|------|
| Sin estacionalidad | None | — | 22.82 | 16.25 | 0.549 |
| **Con estacionalidad aditiva ** | **'add'** | **52** | **18.87** | **12.79** | **0.454** |

**Conclusión:**  
El modelo **semanal con estacionalidad aditiva (m=52)** logra el mejor desempeño:
- Disminuye el error relativo (**sMAPE −21 %**) y mejora la eficiencia (**MASE −17 %**) frente al modelo sin estacionalidad.  
- Captura correctamente el patrón anual semanal.

**Configuración recomendada:**
```python
trend='add', damped_trend=True, seasonal='add', seasonal_periods=52

##  Modelo Mensual – ETS (Holt–Winters)

Se evaluaron configuraciones del modelo **ETS (Error–Trend–Seasonal)** para la serie **mensual** de nuevos casos, variando la presencia o ausencia de estacionalidad.

**Parámetros comunes:**  
`trend='add'`, `damped_trend=True`, `seasonal_periods=12`

---

### 🔧 Configuraciones evaluadas

| Configuración | seasonal | seasonal_periods | MAE | sMAPE (%) | MASE |
|---------------|-----------|------------------|-----|-----------|------|
| **Sin estacionalidad ** | **None** | — | **114.48** | **17.48** | **0.651** |
| Con estacionalidad aditiva | 'add' | 12 | 137.34 | 20.42 | 0.781 |

---

###  Análisis de resultados

- El modelo **sin estacionalidad** obtiene los **menores errores relativos**:  
  - **sMAPE = 17.48 %** → error medio moderado en proporción al valor real.  
  - **MASE = 0.651** → el modelo es **más eficiente** que un modelo ingenuo (MASE < 1).  
- La **estacionalidad aditiva** (m = 12) **empeora el rendimiento**, indicando que el patrón mensual no es lo suficientemente estable o fuerte.  

---

###  Configuración recomendada
```python
trend='add', damped_trend=True, seasonal=None, seasonal_periods=12


#  Comparación de Modelos SARIMA (Semanal vs Mensual)

Se evaluaron modelos **SARIMA** variando órdenes **(p,d,q)** y **(P,D,Q,S)** para dos frecuencias temporales: **semanal** y **mensual**.  
Las métricas de comparación son **MAE** (no comparable entre frecuencias), **sMAPE (%)** y **MASE** (comparable al normalizar contra un naïve estacional).

---

##  Modelo Semanal — SARIMA

**Parámetros/ideas base:**  
- Diferenciación regular: `d = 0` (ETS rindió bien sin diferenciar).  
- Estacional: `D = 1`, `S = 52` (patrón anual semanal).  
- Órdenes cortos en p/q y un **MA estacional** suelen ser efectivos.

| Configuración | order (p,d,q) | seasonal_order (P,D,Q,S) | MAE | sMAPE (%) | MASE |
|---|---|---|---:|---:|---:|
| Base | (1,0,1) | (0,1,1,52) | 17.75 | 11.97 | 0.427 |
| **Ajustado (mejor)**  | **(2,0,1)** | **(0,1,1,52)** | **17.51** | **11.82** | **0.422** |

**Conclusión (semanal):**  
El ajuste a **SARIMA(2,0,1)×(0,1,1)\_52** mejora levemente al base y **supera a ETS** semanal.  
- **sMAPE ↓** (11.97 → **11.82**), **MASE ↓** (0.427 → **0.422**).  
- Mantener `D=1, S=52`, con orden corto y **MA estacional**.

**Configuración recomendada (semanal):**
```python
order=(2,0,1), seasonal_order=(0,1,1,52)

#  Conclusión de Modelo ARIMA + Fourier (Semanal)

Se evaluó **ARIMA con términos de Fourier** para capturar la estacionalidad **semanal** (m=52), ajustando órdenes **(p,d,q)** y el número de armónicos **K**.

---

##  Modelo Semanal — ARIMA + Fourier

**Tipo:** ARIMA con estacionalidad determinística vía Fourier  
**Parámetros base:** `m=52` (semanal)  
**Hiperparámetros evaluados:** `(p,d,q)` en { (1,0,1), (2,0,1), (1,0,2) } y `K ∈ {1, 2}`

| Configuración | K | m | MAE | sMAPE (%) | MASE |
|---|---:|---:|---:|---:|---:|
| **order=(2,0,1)** | **2** | **52** | **17.68** | **12.42** | **0.426** |
| order=(1,0,2) | 2 | 52 | 18.58 | 13.09 | 0.447 |
| order=(1,0,1) | 1 | 52 | 23.70 | 16.99 | 0.571 |
| order=(2,0,1) | 1 | 52 | 20.64 | 14.62 | 0.497 |

---

###  Conclusión
La mejor configuración **ARIMA + Fourier** semanal es:
- **order=(2,0,1), K=2, m=52**  
- Obtiene **sMAPE = 12.42%** y **MASE = 0.426**, superando a otras variantes con K=1 o con órdenes más simples.

> Nota: Aunque este AF es muy competitivo, aún queda **ligeramente por detrás** del **SARIMA semanal** óptimo que probaste (sMAPE ≈ 11.82%, MASE ≈ 0.422).

---

###  Configuración recomendada
```python
res_af, fc_af = run_model("ARIMA_FOURIER", order=(2,0,1), K=2, m=52)


#  Resumen de UCM (Semanal vs Mensual)

Se evaluaron **UCM (Unobserved Components Models)** variando el componente de **nivel** y la **estacionalidad** para series **semanales** y **mensuales**. Métricas clave: **MAE** (no comparable entre frecuencias), **sMAPE (%)** y **MASE**.

---

##  UCM Semanal

**Parámetros probados:**  
- `level ∈ {'local level', 'local linear trend'}`  
- `seasonal = 52` (estacionalidad anual semanal)

| level               | seasonal | MAE   | sMAPE (%) | MASE  |
|---------------------|----------|------:|----------:|------:|
| **local level**   | **52**   | **17.88** | **12.16** | **0.430** |
| local linear trend  | 52       | 19.61 | 13.20 | 0.472 |

**Conclusión (semanal):**  
La versión **simple** (`local level`) con estacionalidad `52` es la mejor dentro de UCM.  
Aunque es **decente** (MASE < 1), queda **ligeramente por detrás** del SARIMA semanal óptimo (≈ 11.82% sMAPE, 0.422 MASE).

**Configuración recomendada (semanal):**
```python
res_ucm, fc_ucm = run_model("UCM", level="local level", seasonal=52)


#  Conclusión de STL + ARIMA (Semanal vs Mensual)

Se evaluó el modelo **STL + ARIMA**, que combina la descomposición estacional-trend mediante *Seasonal-Trend decomposition using Loess (STL)* y un modelo **ARIMA** sobre el residuo desestacionalizado.  
El objetivo fue analizar su desempeño tanto en frecuencia **semanal (m=52)** como **mensual (m=12)**.

---

##  STL + ARIMA Semanal

**Configuraciones evaluadas:**  
- `(period=52, order=(1,0,1))`  
- `(period=52, order=(1,1,1))`

| period | order | MAE  | sMAPE (%) | MASE  |
|---------|---------|------:|----------:|------:|
| **52** | **(1,0,1)**  | **28.62** | **18.02** | **0.689** |
| 52 | (1,1,1) | 30.18 | 18.80 | 0.727 |

**Conclusión (semanal):**  
El mejor resultado se obtuvo con `order=(1,0,1)` y `period=52`.  
Sin embargo, **su desempeño es inferior** al resto de los modelos evaluados:  
- SARIMA (2,0,1)×(0,1,1,52) → **sMAPE ≈ 11.82%, MASE ≈ 0.422**  
- UCM (local level, seasonal=52) → **sMAPE ≈ 12.16%, MASE ≈ 0.430**

**Configuración recomendada (referencial):**
```python
res_stl, fc_stl = run_model("STL_ARIMA", period=52, order=(1,0,1))


# Resumen — Búsqueda local SARIMA con shocks (exógenas)

**Objetivo:**  
Optimizar un modelo **SARIMA** semanal (S=52) incorporando **shocks exógenos** (dummies) y probando dos transformaciones (`log`, `log1p`).  

**Métricas de evaluación:**  
- MAE (Error Absoluto Medio)  
- sMAPE (Error Porcentual Absoluto Simétrico)  
- MASE (Error Escalado Absoluto Medio)  

**Alcance:**  
Se evaluaron **108 combinaciones válidas** de hiperparámetros sin errores de ejecución.

---

## Top-10 combinaciones (ordenadas por sMAPE)

| # | order | seasonal_order | transform | MAE | sMAPE | MASE |
|---:|:----:|:--------------:|:---------:|----:|------:|-----:|
| 1 | (3,0,2) | (0,1,1,52) | log1p | **16.5023** | **11.2569%** | **0.3974** |
| 2 | (3,0,0) | (1,1,2,52) | log | 16.4570 | 11.3263% | 0.3963 |
| 3 | (3,0,0) | (0,1,1,52) | log1p | 16.5685 | 11.3321% | 0.3990 |
| 4 | (3,0,0) | (1,1,2,52) | log1p | 16.4966 | 11.3519% | 0.3972 |
| 5 | (3,0,0) | (0,1,1,52) | log | 16.6000 | 11.3534% | 0.3997 |
| 6 | (2,0,2) | (1,1,2,52) | log | 16.6204 | 11.3752% | 0.4002 |
| 7 | (2,0,2) | (0,1,1,52) | log | 16.7513 | 11.3966% | 0.4034 |
| 8 | (2,0,2) | (0,1,1,52) | log1p | 16.7637 | 11.4021% | 0.4037 |
| 9 | (3,0,2) | (0,1,1,52) | log | 16.7487 | 11.4102% | 0.4033 |
| 10 | (2,0,2) | (1,1,2,52) | log1p | 16.6788 | 11.4310% | 0.4016 |

---

##  Mejor configuración seleccionada

- **Modelo:** SARIMA con shocks (dummies exógenas)  
- **Parámetros:**  
  `order = (3,0,2)`  
  `seasonal_order = (0,1,1,52)`  
  `transform = "log1p"`  
- **Resultados:**  
  - **MAE = 16.5023**  
  - **sMAPE = 11.2569%**  
  - **MASE = 0.3974**